In [1]:
from dash import Dash,html,dcc,Input,Output
from jupyter_dash import JupyterDash
#from dash_bootstrap_components import themes
import dash_bootstrap_components as dbc
from datetime import date
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
#!pip install dash-bootstrap-components

Reading the super_19_stage3 csv file 

In [2]:
covid19_data = pd.read_csv('../../../data/Stage_2/super_19_stage3.csv')
covid19_data

,countyFIPS,County Name,State,StateFIPS,Dates,cases,deaths,population
0,1001,Autauga County,AL,1,2020-01-22,0,0,55869
1,1001,Autauga County,AL,1,2020-01-23,0,0,55869
2,1001,Autauga County,AL,1,2020-01-24,0,0,55869
3,1001,Autauga County,AL,1,2020-01-25,0,0,55869
4,1001,Autauga County,AL,1,2020-01-26,0,0,55869
...,...,...,...,...,...,...,...,...
3427917,56045,Weston County,WY,56,2023-01-12,1881,22,6927
3427918,56045,Weston County,WY,56,2023-01-13,1881,22,6927
3427919,56045,Weston County,WY,56,2023-01-14,1881,22,6927
3427920,56045,Weston County,WY,56,2023-01-15,1881,22,6927


In [3]:
#Converting the Dates column type to Datetime
covid19_data['Dates'] = pd.to_datetime(covid19_data['Dates'])
#Selecting the date range from May 31st 2022 to Jan 1st 2023
covid19_data = covid19_data[(covid19_data['Dates'] >= '2022-05-31') & (covid19_data['Dates'] < '2023-01-01')]

usa_data = covid19_data.groupby(covid19_data['Dates']).sum().reset_index() # Grouping the dataframe by the Dates column
states_data = covid19_data.groupby(['State','Dates']).sum().reset_index() # Grouping the dataframe by State and Dates column
 
usa_data['new_cases'] = usa_data['cases'].diff() #Finding the new cases by using diff() function
usa_data['new_deaths'] = usa_data['deaths'].diff() #Finding the new deaths by using diff() function

usa_data['new_cases'].clip(lower=0, inplace = True) #Clipping new_cases to zero
usa_data['new_deaths'].clip(lower=0, inplace = True) #Clipping new_deaths to zero


 
states_data['new_cases'] = states_data['cases'].diff() #Finding the new cases for each state by using diff() function
states_data['new_deaths'] = states_data['deaths'].diff() #Finding the new deaths for each state by using diff() function

states_data['new_cases'].clip(lower=0, inplace = True) #Clipping new_cases to zero
states_data['new_deaths'].clip(lower=0, inplace = True) #Clipping new_deaths to zero

#Removing the rows whose date is May 31st 2022
usa_data= usa_data[usa_data['Dates']!= '2022-05-31']
states_data= states_data[states_data['Dates']!= '2022-05-31']

#Dropping the unwanted columns
usa_data = usa_data.drop(['countyFIPS','StateFIPS'],axis=1)
states_data = states_data.drop(['countyFIPS','StateFIPS'],axis=1)

usa_data

,Dates,cases,deaths,population,new_cases,new_deaths
1,2022-06-01,81427445,946824,328239523,158949.0,483.0
2,2022-06-02,81494654,947016,328239523,67209.0,192.0
3,2022-06-03,81701504,947235,328239523,206850.0,219.0
4,2022-06-04,81712058,947279,328239523,10554.0,44.0
5,2022-06-05,81737066,947279,328239523,25008.0,0.0
...,...,...,...,...,...,...
210,2022-12-27,93510693,990276,328239523,18100.0,75.0
211,2022-12-28,93538893,990493,328239523,28200.0,217.0
212,2022-12-29,93619173,990707,328239523,80280.0,214.0
213,2022-12-30,93726757,992217,328239523,107584.0,1510.0


In [4]:
#Normalizing the new_cases and new_deaths by population
usa_data['Normalized cases'] = (usa_data['new_cases']/usa_data['population'])*100000
usa_data['Normalized deaths'] = (usa_data['new_deaths']/usa_data['population'])*100000

#Clipping the lower values of normalized cases and normalized deaths to zero
usa_data['Normalized cases'].clip(lower=0, inplace = True) 
usa_data['Normalized deaths'].clip(lower=0, inplace = True)

#Finding the log normalized cases and log normalized deaths by using log from numpy
usa_data['Log normalized cases']= np.log(usa_data['Normalized cases'])
usa_data['Log normalized deaths']= np.log(usa_data['Normalized deaths'])

#Clipping the lower values of log normalized cases and log normalized deaths to zero
usa_data['Log normalized cases'].clip(lower=0, inplace = True) 
usa_data['Log normalized deaths'].clip(lower=0, inplace = True) 

In [5]:
states_data['Normalized cases'] = (states_data['new_cases']/states_data['population'])*100000
states_data['Normalized deaths'] = (states_data['new_deaths']/states_data['population'])*100000

states_data['Normalized cases'].clip(lower=0, inplace = True) 
states_data['Normalized deaths'].clip(lower=0, inplace = True)

states_data['Log normalized cases']= np.log(states_data['Normalized cases'])
states_data['Log normalized deaths']= np.log(states_data['Normalized deaths'])

states_data['Log normalized cases'].clip(lower=0, inplace = True) 
states_data['Log normalized deaths'].clip(lower=0, inplace = True)

In [6]:
usa_data

,Dates,cases,deaths,population,new_cases,new_deaths,Normalized cases,Normalized deaths,Log normalized cases,Log normalized deaths
1,2022-06-01,81427445,946824,328239523,158949.0,483.0,48.424699,0.147149,3.880010,0.0
2,2022-06-02,81494654,947016,328239523,67209.0,192.0,20.475596,0.058494,3.019234,0.0
3,2022-06-03,81701504,947235,328239523,206850.0,219.0,63.018005,0.066720,4.143420,0.0
4,2022-06-04,81712058,947279,328239523,10554.0,44.0,3.215335,0.013405,1.167932,0.0
5,2022-06-05,81737066,947279,328239523,25008.0,0.0,7.618827,0.000000,2.030622,0.0
...,...,...,...,...,...,...,...,...,...,...
210,2022-12-27,93510693,990276,328239523,18100.0,75.0,5.514266,0.022849,1.707339,0.0
211,2022-12-28,93538893,990493,328239523,28200.0,217.0,8.591287,0.066110,2.150749,0.0
212,2022-12-29,93619173,990707,328239523,80280.0,214.0,24.457749,0.065196,3.196947,0.0
213,2022-12-30,93726757,992217,328239523,107584.0,1510.0,32.776065,0.460030,3.489699,0.0


In [7]:
states_data

,State,Dates,cases,deaths,population,new_cases,new_deaths,Normalized cases,Normalized deaths,Log normalized cases,Log normalized deaths
1,AK,2022-06-01,244906,1231,731545,0.0,0.0,0.0,0.0,0.0,0.0
2,AK,2022-06-02,244906,1231,731545,0.0,0.0,0.0,0.0,0.0,0.0
3,AK,2022-06-03,244906,1231,731545,0.0,0.0,0.0,0.0,0.0,0.0
4,AK,2022-06-04,244906,1231,731545,0.0,0.0,0.0,0.0,0.0,0.0
5,AK,2022-06-05,244906,1231,731545,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
10960,WY,2022-12-27,182581,1958,578759,0.0,0.0,0.0,0.0,0.0,0.0
10961,WY,2022-12-28,182581,1958,578759,0.0,0.0,0.0,0.0,0.0,0.0
10962,WY,2022-12-29,182581,1958,578759,0.0,0.0,0.0,0.0,0.0,0.0
10963,WY,2022-12-30,182581,1958,578759,0.0,0.0,0.0,0.0,0.0,0.0


Function which accepts a dataframe and column name and returns the dataframe which contains Dates, first_day columns. It is similar to the function which is used in stage 4

In [8]:
def first_day_new_cases_deaths(df,col):
    new_cases_deaths_list = df[col].tolist()
    first_day_list = []
    count = 1
    for idx,i in enumerate(new_cases_deaths_list):
        if(int(i)>=1):
            ind = idx #Finding the index
            break
    for j in range(len(new_cases_deaths_list)):
        if(j>=ind):
            first_day_list.append(count)  #Appending the count from the first day of infection
            count = count+1
        else:
            first_day_list.append(0)
    firstday_df = df[['Dates',col]]
    firstday_df['first_day'] = first_day_list
    firstday_df = firstday_df.iloc[ind:].reset_index(drop=True)
    return firstday_df 

This function is similar to the above function but in this function it calculates for each state present in the data frame.

In [9]:
def first_day_new_cases_deaths_states(df,col):
    states_list = df['State'].unique().tolist()
    c = df.State.value_counts()
    firstday_final_df = pd.DataFrame()
    for i in range(len(c)):
        df1 = df[df['State']==states_list[i]]
        new_cases_deaths_list = df1[col].tolist()
        first_day_list = []
        count = 1
        for idx,i in enumerate(new_cases_deaths_list):
            if(int(i)>=1):
                ind = idx #Finding the index
                break
        for j in range(len(new_cases_deaths_list)):
            if(j>=ind):
                first_day_list.append(count) #Appending the count from the first day of infection
                count = count+1
            else:
                first_day_list.append(0)
        firstday_df = df1[['State','Dates',col]]
        firstday_df['first_day'] = first_day_list
        firstday_df = firstday_df.iloc[ind:].reset_index(drop=True)
        firstday_final_df = pd.concat([firstday_final_df,firstday_df])
    return firstday_final_df 

This is the function which is used to find the regression models  (Linear and Polynomial) trend line for the data across USA.

In [10]:
def regression(df,col,deg):
    X = np.stack(df['first_day']).reshape(-1,1)
    y = np.stack(df[col]).reshape(-1,1)
    if(deg == 1):
        lm=LinearRegression() #Linear Regression Model
        lm.fit(X,y)
        y_pred=lm.predict(X)
    elif(deg>=2):
        poly_reg = PolynomialFeatures(degree=deg)  #Polynomial Regression Model
        X_poly = poly_reg.fit_transform(X)
        lin_reg2 = LinearRegression()
        lin_reg2.fit(X_poly,y)
        y_pred = lin_reg2.predict(X_poly)
    final_df = pd.DataFrame(X)
    final_df.rename({0: 'first_day'}, axis=1, inplace=True)
    temp1 = pd.DataFrame(y)
    temp2 = pd.DataFrame(y_pred)
    final_df['actual']= temp1[0]
    final_df['predicted']= temp2[0]
    return final_df

This function is similar to the above regression function but used to find the regression models trend line for each state present in dataframe 

In [11]:
def regression_states(df,col,deg):
    states_list = df['State'].unique().tolist()
    c = df.State.value_counts()
    reg_final_df = pd.DataFrame()
    #final_df = pd.DataFrame()
    for i in range(len(c)):
        df1 = df[df['State']==states_list[i]]
        
        X = np.stack(df1['first_day']).reshape(-1,1)
        y = np.stack(df1[col]).reshape(-1,1)
        if(deg == 1):
            lm=LinearRegression() # Linear Regression Model
            lm.fit(X,y)
            y_pred=lm.predict(X)
        elif(deg>=2):
            poly_reg = PolynomialFeatures(degree=deg)  #Polynomial Regression Model
            X_poly = poly_reg.fit_transform(X)
            lin_reg2 = LinearRegression()
            lin_reg2.fit(X_poly,y)
            y_pred = lin_reg2.predict(X_poly)
        final_df = df1[['State']]
        temp1 = pd.DataFrame(X)
        #final_df.rename({0: 'first_day'}, axis=1, inplace=True)
        temp2 = pd.DataFrame(y)
        temp3 = pd.DataFrame(y_pred)
        final_df['first_day'] = temp1[0]
        final_df['actual']= temp2[0]
        final_df['predicted']= temp3[0]
        reg_final_df = pd.concat([reg_final_df,final_df])
        
    return reg_final_df

Function which calculates the moving average for the states selected

In [12]:
def moving_avg_states(df,col):
    states_list = df['State'].unique().tolist() # Finding the unique states
    c = df.State.value_counts()
    final_df = pd.DataFrame()
    for i in range(len(c)):
        df1 = df[df['State']==states_list[i]]
        df1['7day_avg']= df1[col].rolling(window=7).mean()
        final_df = pd.concat([final_df,df1])
    return final_df
        
    
    

In [13]:
app = JupyterDash(__name__)
# Structure of the dash
app.layout = html.Div([
    # Main heading of the DASH
    html.H1("Covid Data Analysis",style = {'text-align':'center'}),
    
    html.Br(),
    
    # Date Range picker
     html.Div(className='row',children =[
    html.Div([
        html.Label(['Date Range'], style={'font-weight': 'bold', "text-align": "right","offset":1}),
        html.Br(),
        dcc.DatePickerRange(
        id='my-date-picker-range',
        start_date = date(2022, 6, 1),
        min_date_allowed=date(2022, 6, 1),
        max_date_allowed=date(2022, 12, 31),
        initial_visible_month=date(2022, 6, 1),
        end_date=date(2022, 12, 31),
        #style = {'width':'25%'}
    ),
    ],style = {'width':'20%','text-align':'center'}),
    ],style = dict(display = 'flex',justifyContent = 'center')),
    
    html.Br(),
    
    # Choosing the parameter
    html.Div(className='row',children =[
    html.Div([
        html.Label(['Cases/Deaths'], style={'font-weight': 'bold', "text-align": "right","offset":1}),
        dcc.Dropdown(id = 'parameter',
       options=[
           {'label': 'Cases', 'value': 'Cases'},
           {'label': 'Deaths', 'value': 'Deaths'},],
       multi = False,
       value= 'Cases',
       ),  
        
    ],style = {'width':'20%','text-align':'center'}),   
        
 ],style = dict(display = 'flex',justifyContent = 'center')),
 
    
    html.Br(),
    
 
    #Dropdown to select for the States
    html.Div(className='row',children =[
    html.Div([
        html.Label(['State'], style={'font-weight': 'bold', "text-align": "right","offset":1}),
        dcc.Dropdown(id='state',   
           options=[
            {'label':'AK' , 'value':'AK' },
            {'label':'AL' , 'value':'AL' },
            {'label':'AR' , 'value':'AR' },
            {'label':'AZ' , 'value':'AZ' },
            {'label':'CA' , 'value':'CA' },
            {'label':'CO' , 'value':'CO' },
            {'label':'CT' , 'value':'CT' },
            {'label':'DC' , 'value':'DC' },
            {'label':'DE' , 'value':'DE' },
            {'label':'FL' , 'value':'FL' },
            {'label':'GA' , 'value':'GA' },
            {'label':'HI' , 'value':'HI' },
            {'label':'IA' , 'value':'IA' },
            {'label':'ID' , 'value':'ID' },
            {'label':'IL' , 'value':'IL' },
            {'label':'IN' , 'value':'IN' },
            {'label':'KS' , 'value':'KS' },
            {'label':'KY' , 'value':'KY' },
            {'label':'LA' , 'value':'LA' },
            {'label':'MA' , 'value':'MA' },
            {'label':'MD' , 'value':'MD' },
            {'label':'ME' , 'value':'ME' },
            {'label':'MI' , 'value':'MI' },
            {'label':'MN' , 'value':'MN' },
            {'label':'MO' , 'value':'MO' },
            {'label':'MS' , 'value':'MS' },
            {'label':'MT' , 'value':'MT' },
            {'label':'NC' , 'value':'NC' },
            {'label':'ND' , 'value':'ND' },
            {'label':'NE' , 'value':'NE' },
            {'label':'NH' , 'value':'NH' },
            {'label':'NJ' , 'value':'NJ' },
            {'label':'NM' , 'value':'NM' },
            {'label':'NV' , 'value':'NV' },
            {'label':'NY' , 'value':'NY' },
            {'label':'OH' , 'value':'OH' },
            {'label':'OK' , 'value':'OK' },
            {'label':'OR' , 'value':'OR' },
            {'label':'PA' , 'value':'PA' },
            {'label':'RI' , 'value':'RI' },
            {'label':'SC' , 'value':'SC' },
            {'label':'SD' , 'value':'SD' },
            {'label':'TN' , 'value':'TN' },
            {'label':'TX' , 'value':'TX' },
            {'label':'UT' , 'value':'UT' },
            {'label':'VA' , 'value':'VA' },
            {'label':'VT' , 'value':'VT' },
            {'label':'WA' , 'value':'WA' },
            {'label':'WI' , 'value':'WI' },
            {'label':'WV' , 'value':'WV' },
            {'label':'WY' , 'value':'WY' },],
                multi = True,
                value = None,
                #style = {'width':'35%','text-align':'center'}
                ),],style = {'width':'20%','text-align':'center'}),
        ],style = dict(display = 'flex',justifyContent = 'center')),
        
    
 
   html.Br(),
    
    #Linear or Log Normalized
    html.Div(className='row',children =[
    html.Div([
        html.Label([''], style={'font-weight': 'bold', "text-align": "right","offset":1}),
        dcc.RadioItems(id='radio',  
           options=[
           {'label': 'Linear', 'value': 0},
           #{'label': 'Normalized', 'value': 1},
           {'label': 'Log normalized', 'value': 2}
        ],
        value=0
        ),],style = {'width':'20%','text-align':'center'}),
        ],style = dict(display = 'flex',justifyContent = 'center')),
        
    
    html.Br(),
    #Regression
    html.Div(className='row',children =[
    html.Div([
        html.Label(['Regression'], style={'font-weight': 'bold', "text-align": "right","offset":1}),
        dcc.Dropdown(id='deg',    
           options=[
           {'label': 'Linear Regression', 'value': 1},
           {'label': 'Non Linear Regression with degree 2', 'value': 2},
           {'label': 'Non Linear Regression with degree 3', 'value': 3},
           {'label': 'Non Linear Regression with degree 4', 'value': 4},
           {'label': 'Non Linear Regression with degree 5', 'value': 5},
        ],
        multi = False,
        value = None,
        #style = {'width':'40%','text-align':'center'}
        ),],style = {'width':'40%','text-align':'center'}),
        ],style = dict(display = 'flex',justifyContent = 'center')),
        
    
    html.Br(),
    
    #7day moving average
    html.Div(className='row',children =[
    html.Div([
        dcc.RadioItems(id='days', 
           options=[
               {'label': '7-days moving average', 'value': 1},
            ],
            value= None
        ),],style = {'width':'20%','text-align':'center'}),
        
    ],style = dict(display = 'flex',justifyContent = 'center')),
    
     html.Br(),
    
    
    html.Div(className='row',children =[
    html.Div(id='output_container',children = []),
       ],style = dict(display = 'flex',justifyContent = 'center')),  #Output container

      
    
    dcc.Graph(id='trends', figure={}) # Graph output
    
    #dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal')
])

In [14]:
#Callback of dash when inputs are selected by the user
@app.callback(
    # Output Components 
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='trends', component_property='figure')],
    # Input Components
    [Input(component_id='parameter', component_property='value'), 
     Input('my-date-picker-range', 'start_date'),
     Input('my-date-picker-range', 'end_date'),
     Input(component_id='state', component_property='value'),
     Input(component_id='radio', component_property='value'),
     Input(component_id='deg', component_property='value'),
     Input(component_id='days', component_property='value'),
    ]
)
def update_graph(parameter,start_date,end_date,state,radio,deg,days):
    
    output_container = 'Selected {} from {} to {} across the {} state '.format(parameter,start_date,end_date,state)
    if(state): # IF State is selected
        df_states = states_data.copy()
        mask = (df_states['Dates'] >= start_date) & (df_states['Dates'] <= end_date)
        df_states = df_states.loc[mask]
        df_states=df_states[df_states['State'].isin(state)]
        if(parameter == 'Cases'):
            fig = px.line(df_states,x = 'Dates',y = 'new_cases',color= 'State',title = 'Cases across selected States'+str(state))
            if(deg):
                first_day = first_day_new_cases_deaths_states(df_states,'new_cases')
                final_df = regression_states(first_day,'new_cases',deg)
                fig1 = px.line(final_df, x='first_day',y='predicted',color = 'State')
                #fig1.update_traces(line_color='red', line_width=3)
                fig2 = px.line(final_df, x='first_day',y='actual',color = 'State')
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Days from first day of infection',yaxis_title = 'Actual and predicted cases',title = 'Model predicted trend line for new cases across States'+str(state))
            elif(days):
                moving_average = moving_avg_states(df_states,'new_cases')
                fig1 = px.line(moving_average, x='Dates',y='new_cases',color = 'State')
                fig2 = px.line(moving_average, x='Dates',y='7day_avg',color = 'State')
                #fig2.update_traces(line_color='red', line_width=3)
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'New cases and 7day moving average',title = '7day Moving average for New Cases across States'+str(state))
                
        if(parameter == 'Cases' and radio == 1):
            fig = px.line(df_states,x = 'Dates',y = 'Normalized cases',color= 'State',title = 'Normalized Cases across selected States'+str(state))
            if(deg):
                first_day = first_day_new_cases_deaths_states(df_states,'Normalized cases')
                final_df = regression_states(first_day,'Normalized cases',deg)
                fig1 = px.line(final_df, x='first_day',y='predicted',color = 'State')
                #fig1.update_traces(line_color='red', line_width=3)
                fig2 = px.line(final_df, x='first_day',y='actual',color = 'State')
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Days from first day of infection',yaxis_title = 'Actual and predicted cases',title = 'Model predicted trend line for Normalized cases across States'+str(state))
            elif(days):
                moving_average = moving_avg_states(df_states,'Normalized cases')
                fig1 = px.line(moving_average, x='Dates',y='Normalized cases',color = 'State')
                fig2 = px.line(moving_average, x='Dates',y='7day_avg',color = 'State')
                #fig2.update_traces(line_color='red', line_width=3)
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'Normalized cases and 7day moving average',title = '7day Moving average for Normalized Cases across States'+str(state))
        
        if(parameter == 'Cases'and radio == 2):
            fig = px.line(df_states,x = 'Dates',y = 'Log normalized cases',color= 'State',title = 'Log Normalized Cases across selected States'+str(state))
            if(deg):
                first_day = first_day_new_cases_deaths_states(df_states,'Log normalized cases')
                final_df = regression_states(first_day,'Log normalized cases',deg)
                fig1 = px.line(final_df, x='first_day',y='predicted',color = 'State')
                #fig1.update_traces(line_color='red', line_width=3)
                fig2 = px.line(final_df, x='first_day',y='actual',color = 'State')
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Days from first day of infection',yaxis_title = 'Actual and predicted cases',title = 'Model predicted trend line for Log Normalized Cases across States'+str(state))
            elif(days):
                moving_average = moving_avg_states(df_states,'Log normalized cases')
                fig1 = px.line(moving_average, x='Dates',y='Log normalized cases',color = 'State')
                fig2 = px.line(moving_average, x='Dates',y='7day_avg',color = 'State')
                #fig2.update_traces(line_color='red', line_width=3)
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'Log Normalized cases and 7day moving average',title = '7day Moving average for Log Normalized Cases across States'+str(state))
                
        if(parameter == 'Deaths'):
            fig = px.line(df_states,x = 'Dates',y = 'new_deaths',color= 'State',title = 'Dases across selected States'+str(state))
            if(deg):
                first_day = first_day_new_cases_deaths_states(df_states,'new_deaths')
                final_df = regression_states(first_day,'new_deaths',deg)
                fig1 = px.line(final_df, x='first_day',y='predicted',color = 'State')
                #fig1.update_traces(line_color='red', line_width=3)
                fig2 = px.line(final_df, x='first_day',y='actual',color = 'State')
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Days from first day of deaths',yaxis_title = 'Actual and predicted deaths',title = 'Model predicted trend line for new deaths across States'+str(state))
            elif(days):
                moving_average = moving_avg_states(df_states,'new_deaths')
                fig1 = px.line(moving_average, x='Dates',y='new_deaths',color = 'State')
                fig2 = px.line(moving_average, x='Dates',y='7day_avg',color = 'State')
                #fig2.update_traces(line_color='red', line_width=3)
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'New deaths and 7day moving average',title = '7day Moving average for New Deaths across States'+str(state))
        
        if(parameter == 'Deaths' and radio == 1):
            fig = px.line(df_states,x = 'Dates',y = 'Normalized deaths',color= 'State',title = 'Normalized Deaths across selected States'+str(state))
            if(deg):
                first_day = first_day_new_cases_deaths_states(df_states,'Normalized deaths')
                final_df = regression_states(first_day,'Normalized deaths',deg)
                fig1 = px.line(final_df, x='first_day',y='predicted',color = 'State')
                #fig1.update_traces(line_color='red', line_width=3)
                fig2 = px.line(final_df, x='first_day',y='actual',color = 'State')
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Days from first day of deaths',yaxis_title = 'Actual and predicted normalized deaths',title = 'Model predicted trend line for Normalized deaths across States'+str(state))
            elif(days):
                moving_average = moving_avg_states(df_states,'Normalized deaths')
                fig1 = px.line(moving_average, x='Dates',y='Normalized deaths',color = 'State')
                fig2 = px.line(moving_average, x='Dates',y='7day_avg',color = 'State')
                #fig2.update_traces(line_color='red', line_width=3)
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'Normalized deaths and 7day moving average',title = '7day Moving average for Normalized Deaths across States'+str(state))
                
        if(parameter == 'Deaths'and radio == 2):
            fig = px.line(df_states,x = 'Dates',y = 'Log normalized deaths',color= 'State',title = 'Log Normalized Deaths across selected States'+str(state))
            if(deg):
                first_day = first_day_new_cases_deaths_states(df_states,'Log normalized deaths')
                final_df = regression_states(first_day,'Log normalized deaths',deg)
                fig1 = px.line(final_df, x='first_day',y='predicted',color = 'State')
                #fig1.update_traces(line_color='red', line_width=3)
                fig2 = px.line(final_df, x='first_day',y='actual',color = 'State')
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Days from first day of deaths',yaxis_title = 'Actual and predicted deaths',title = 'Model predicted trend line for Log Normalized Deaths across States'+str(state))
            elif(days):
                moving_average = moving_avg_states(df_states,'Log normalized deaths')
                fig1 = px.line(moving_average, x='Dates',y='Log normalized deaths',color = 'State')
                fig2 = px.line(moving_average, x='Dates',y='7day_avg',color = 'State')
                #fig2.update_traces(line_color='red', line_width=3)
                fig = go.Figure(data=fig1.data + fig2.data)
                fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'Log Normalized deaths and 7day moving average',title = '7day Moving average for Log Normalized Deaths across States'+str(state))
                
        
        
        
        
        
            
                
        
        
        
        return output_container,fig
        
        
        
        
        
        
        
        
        
        
        
    # For entire usa data    
    df_usa = usa_data.copy()
    mask = (df_usa['Dates'] >= start_date) & (df_usa['Dates'] <= end_date) # Selected date range
    df_usa = df_usa.loc[mask]
    #usa_data = usa_data[(usa_data['Dates'] >= start_date) & (usa_data['Dates'] <= end_date)]
    if(parameter == 'Cases'):
        fig = px.line(df_usa,x = 'Dates',y = 'new_cases',title = 'Cases across United States of America')
        if(deg):
            first_day = first_day_new_cases_deaths(df_usa,'new_cases')
            final_df = regression(first_day,'new_cases',deg)
            fig1 = px.line(final_df, x='first_day',y='predicted')
            fig1.update_traces(line_color='red', line_width=3)
            fig2 = px.line(final_df, x='first_day',y='actual')
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Days from first day of infection',yaxis_title = 'Actual and predicted cases',title = 'Model predicted trend line for new cases across United States of America')
        elif(days):
            df_usa['7day_avg'] = df_usa['new_cases'].rolling(window=7).mean()
            fig1 = px.line(df_usa, x='Dates',y='new_cases')
            fig2 = px.line(df_usa, x='Dates',y='7day_avg')
            fig2.update_traces(line_color='red', line_width=3)
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'New cases and 7day moving average',title = '7day Moving average for New Cases across United States of America')
            
            
            
    if(parameter == 'Cases' and radio == 1):
        fig = px.line(df_usa,x = 'Dates',y = 'Normalized cases',title = 'Normalized Cases across United States of America')
        if(deg):
            first_day = first_day_new_cases_deaths(df_usa,'Normalized cases')
            final_df = regression(first_day,'Normalized cases',deg)
            fig1 = px.line(final_df, x='first_day',y='predicted')
            fig1.update_traces(line_color='red', line_width=3)
            fig2 = px.line(final_df, x='first_day',y='actual')
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Days from first day of infection',yaxis_title = 'Actual and predicted Normalized cases',title = 'Model predicted trend line for Normalized new cases across United States of America')
        elif(days):
            df_usa['7day_avg'] = df_usa['Normalized cases'].rolling(window=7).mean()
            fig1 = px.line(df_usa, x='Dates',y='Normalized cases')
            fig2 = px.line(df_usa, x='Dates',y='7day_avg')
            fig2.update_traces(line_color='red', line_width=3)
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'Normalized cases and 7day moving average',title = '7day Moving average for Normalized Cases across United States of America')
            
    if(parameter == 'Cases'and radio == 2):
        fig = px.line(df_usa,x = 'Dates',y = 'Log normalized cases',title = 'Log Normalized Cases across United States of America')
        if(deg):
            first_day = first_day_new_cases_deaths(df_usa,'Log normalized cases')
            final_df = regression(first_day,'Log normalized cases',deg)
            fig1 = px.line(final_df, x='first_day',y='predicted')
            fig1.update_traces(line_color='red', line_width=3)
            fig2 = px.line(final_df, x='first_day',y='actual')
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Days from first day of infection',yaxis_title = 'Actual and predicted Log Normalized cases',title = 'Model predicted trend line for Log Normalized new cases across United States of America')
        
        elif(days):
            df_usa['7day_avg'] = df_usa['Log normalized cases'].rolling(window=7).mean()
            fig1 = px.line(df_usa, x='Dates',y='Log normalized cases')
            fig2 = px.line(df_usa, x='Dates',y='7day_avg')
            fig2.update_traces(line_color='red', line_width=3)
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'Log normalized cases and 7day moving average',title = '7day Moving average for Log normalized Cases across United States of America')
            
    if(parameter == 'Deaths'):
        fig = px.line(df_usa,x = 'Dates',y = 'new_deaths',title = 'Deaths across United States of America')
        if(deg):
            first_day = first_day_new_cases_deaths(df_usa,'new_deaths')
            final_df = regression(first_day,'new_deaths',deg)
            fig1 = px.line(final_df, x='first_day',y='predicted')
            fig1.update_traces(line_color='red', line_width=3)
            fig2 = px.line(final_df, x='first_day',y='actual')
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Days from first day of deaths',yaxis_title = 'Actual and predicted deaths',title = 'Model predicted trend line for new deaths across United States of America')
        
        elif(days):
            df_usa['7day_avg'] = df_usa['new_deaths'].rolling(window=7).mean()
            fig1 = px.line(df_usa, x='Dates',y='new_deaths')
            fig2 = px.line(df_usa, x='Dates',y='7day_avg')
            fig2.update_traces(line_color='red', line_width=3)
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'New deaths and 7day moving average',title = '7day Moving average for New Deaths across United States of America')
            
    if(parameter == 'Deaths' and radio == 1):
        fig = px.line(df_usa,x = 'Dates',y = 'Normalized deaths',title = 'Normalized Deaths across United States of America')
        if(deg):
            first_day = first_day_new_cases_deaths(df_usa,'Normalized deaths')
            final_df = regression(first_day,'Normalized deaths',deg)
            fig1 = px.line(final_df, x='first_day',y='predicted')
            fig1.update_traces(line_color='red', line_width=3)
            fig2 = px.line(final_df, x='first_day',y='actual')
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Days from first day of deaths',yaxis_title = 'Actual and predicted Normalized deaths',title = 'Model predicted trend line for Normalized new deaths across United States of America')
        
        elif(days):
            df_usa['7day_avg'] = df_usa['Normalized deaths'].rolling(window=7).mean()
            fig1 = px.line(df_usa, x='Dates',y='Normalized deaths')
            fig2 = px.line(df_usa, x='Dates',y='7day_avg')
            fig2.update_traces(line_color='red', line_width=3)
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'Normalized deaths and 7day moving average',title = '7day Moving average for Normalized Deaths across United States of America')
            
    if(parameter == 'Deaths'and radio == 2):
        fig = px.line(df_usa,x = 'Dates',y = 'Log normalized deaths',title = 'Log Normalized Deaths across United States of America')
        if(deg):
            first_day = first_day_new_cases_deaths(df_usa,'Log normalized deaths')
            final_df = regression(first_day,'Log normalized deaths',deg)
            fig1 = px.line(final_df, x='first_day',y='predicted')
            fig1.update_traces(line_color='red', line_width=3)
            fig2 = px.line(final_df, x='first_day',y='actual')
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Days from first day of deaths',yaxis_title = 'Actual and predicted Log Normalized deaths',title = 'Model predicted trend line for Log Normalized new deaths across United States of America')
        
        elif(days):
            df_usa['7day_avg'] = df_usa['Log normalized deaths'].rolling(window=7).mean()
            fig1 = px.line(df_usa, x='Dates',y='Log normalized deaths')
            fig2 = px.line(df_usa, x='Dates',y='7day_avg')
            fig2.update_traces(line_color='red', line_width=3)
            fig = go.Figure(data=fig1.data + fig2.data)
            fig.update_layout(xaxis_title = 'Dates',yaxis_title = 'Log normalized deaths and 7day moving average',title = '7day Moving average for Log normalized Deaths across United States of America')
    
    
    
    
    
    
    
    return output_container,fig

In [15]:
app.run_server(debug= True)

Dash app running on http://127.0.0.1:8050/


In [16]:
#app.run_server(mode='inline')

References:

- https://stackoverflow.com/questions/70916649/how-to-change-the-x-axis-and-y-axis-labels-in-plotly

- https://stackoverflow.com/questions/69026026/change-line-colour-with-plotly-express

- https://dash.plotly.com/dash-core-components/graph

- https://stackoverflow.com/questions/40060842/moving-average-pandas

- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html